<a href="https://colab.research.google.com/github/arutraj/ML_Basics/blob/main/15_6_Transfer_Learning_using_architecture_of_pre_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# %tensorflow_version 1.x
import tensorflow
tensorflow.__version__

'2.15.0'

## Transfer Learning using architecture of pre-trained model

1. Import necessary libraries
2. Load the data
3. Pre-Process the data
4. Load the architecture of pretrained model
5. Train the model
6. Get predictions

## 1. Import neccessary libraries

Here we will import the required libraries and functions.

In [1]:
# import necessary libraries for plotting images
import numpy as np
import matplotlib.pyplot as plt
# plot in jupyter notebook itself
%matplotlib inline
from glob import glob
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from keras.models import Model
from keras.utils import to_categorical
#used to preprocess data according to VGG16
from keras.applications.vgg16 import preprocess_input
#for instantiating the model and loading the weights and biases
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, InputLayer

## 2. Load the data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# unzipping the Dataset.zip file
#!unzip /content/drive/My\ Drive/Dataset.zip
!ls /content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/*jpg |wc -l

2352


In [4]:
#reading the csv file containing data labels
data = pd.read_csv('/content/emergency_classification.csv')

In [5]:
# create an empty python list
X = []

# go through all the image locations one by one
for img_name in data.image_names:
    # read the image from location
    img = plt.imread('/content/drive/MyDrive/AnalyticsVidya/DeepLearning/images/' + img_name)
    # pile it one over the other
    X.append(img)

# convert this python list to a single numpy array
X = np.array(X)

In [6]:
#getting the labels for images
y = data.emergency_or_not.values

## 3. Pre-Process the data

Steps :
1. Pre-process the data as per model's requirement
2. Prepare training and validation set

In [7]:
#preprocess input images accordiing to requirements of VGG16 model
X = preprocess_input(X)

In [8]:
#converting label to categorical i.e instead of 0/1 labels we have 2 columns emergency and non-emergency
# with only one of them is true for every image
y = to_categorical(y)

In [9]:
# splitting the dataset into training and validation sets
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, random_state=42)

## 4. Load the architecture of pretrained model

In [10]:
# creating model with pre trained imagenet weights
base_model = VGG16(weights='imagenet')

553467096/553467096 [==============================] - 3s 0us/step


In [16]:
#shows model summary
base_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [17]:
# creating our own model
x = Dense(100, activation='relu', name='fc1')(base_model.layers[-4].output)
y = Dense(2, activation='softmax', name='prediction')(x)
my_model = Model(base_model.input, y)
my_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [18]:
# compile the model
my_model.compile(optimizer='adam', metrics=['accuracy'], loss='categorical_crossentropy')

## 5. Train the model

In [19]:
# training the model
my_model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
52/52 [==============================] - 59s 666ms/step - loss: 17.6259 - accuracy: 0.5966 - val_loss: 0.7176 - val_accuracy: 0.6062
Epoch 2/10
52/52 [==============================] - 23s 434ms/step - loss: 0.6117 - accuracy: 0.6999 - val_loss: 0.4990 - val_accuracy: 0.7635
Epoch 3/10
52/52 [==============================] - 22s 433ms/step - loss: 0.5194 - accuracy: 0.7527 - val_loss: 0.5635 - val_accuracy: 0.7195
Epoch 4/10
52/52 [==============================] - 23s 435ms/step - loss: 0.5131 - accuracy: 0.7704 - val_loss: 0.4392 - val_accuracy: 0.8059
Epoch 5/10
52/52 [==============================] - 23s 447ms/step - loss: 0.5692 - accuracy: 0.7412 - val_loss: 0.5213 - val_accuracy: 0.7380
Epoch 6/10
52/52 [==============================] - 23s 452ms/step - loss: 0.4891 - accuracy: 0.7728 - val_loss: 0.4790 - val_accuracy: 0.7606
Epoch 7/10
52/52 [==============================] - 23s 445ms/step - loss: 0.4653 - accuracy: 0.7995 - val_loss: 0.4551 - val_accuracy: 0.797

## 6. Get Predictions

In [20]:
# get predictions
predictions = my_model.predict(X_valid)
predicted_class = np.argmax(predictions,axis=1)

23/23 [==============================] - 3s 121ms/step


In [ ]:
# checking the performance of model on validation set
accuracy_score(y_valid[:,1], predicted_class)

0.9405099150141643

## Steps to use the architecture of pre-trained model

1. Import necessary libraries
2. Load the data
3. Pre-Process the data
4. Load the architecture of pretrained model
5. Train the model
6. Get predictions